<a href="https://colab.research.google.com/github/Sarahoin/Project/blob/main/Text_Summarisation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.6 MB/s eta 0:00:00


In [2]:
!pip install transformers datasets fastapi uvicorn rouge-score


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.3/73.3 kB 5.7 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=3979e693ddaa0d4a936266fb71b2a420a98dbc423008b51bcab709ee36240217
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [3]:
from datasets import load_dataset

dataset = load_dataset("cnn_dailymail", '3.0.0')
train_data = dataset['train']
val_data = dataset['validation']
test_data = dataset['test']


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/15.6k [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/34.7M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [ ]:
# Use a smaller portion of the dataset for faster testing
small_train_data = train_data.select(range(1000))  # Select first 1000 examples
small_val_data = val_data.select(range(500))       # Select first 500 examples


In [ ]:
from transformers import BartTokenizer

# Load the tokenizer for the BART model
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')

def preprocess_data(batch):
    # Tokenize the inputs (articles) and labels (highlights)
    inputs = tokenizer(batch['article'], max_length=1024, truncation=True, return_tensors="pt", padding="max_length")
    labels = tokenizer(batch['highlights'], max_length=150, truncation=True, return_tensors="pt", padding="max_length")

    # Remove unnecessary columns and return only input_ids and labels
    batch['input_ids'] = inputs['input_ids'].squeeze()
    batch['attention_mask'] = inputs['attention_mask'].squeeze()
    batch['labels'] = labels['input_ids'].squeeze()

    return batch

# Apply preprocessing to the dataset
train_data = train_data.map(preprocess_data, batched=True)
val_data = val_data.map(preprocess_data, batched=True)

# Remove columns that are not needed for training
train_data.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
val_data.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/287113 [00:00<?, ? examples/s]

Map:   0%|          | 0/13368 [00:00<?, ? examples/s]

In [ ]:
from transformers import BartForConditionalGeneration, BartTokenizer

# Load model and tokenizer
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')


model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

from transformers import Seq2SeqTrainingArguments

# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",  # Change this to `evaluation_strategy` to avoid deprecation warnings
    num_train_epochs=1,
    learning_rate=2e-5,
    weight_decay=0.01,
    save_total_limit=3,
    fp16=True,
    predict_with_generate=True,
    remove_unused_columns=False , # Prevents dropping unused columns
    save_steps=500,  # Save very 500 steps
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4
)


# Trainer setup
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=val_data
)

# Fine-tune the model
trainer.train()

Epoch,Training Loss,Validation Loss


In [ ]:
def summarize_text(text, model, tokenizer, max_length=150, num_beams=4):
    # Tokenize input text
    inputs = tokenizer(text, max_length=1024, return_tensors="pt", truncation=True)

    # Generate summary
    summary_ids = model.generate(inputs['input_ids'], num_beams=num_beams, max_length=max_length, early_stopping=True)

    # Decode and return the summary
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# Example usage
long_text = """The Growing Threat of Climate Change and its Global Impact
Climate change refers to long-term changes in temperature, precipitation, and other atmospheric conditions on Earth. Over the past century, human activities—particularly the burning of fossil fuels like coal, oil, and gas—have significantly increased the concentration of greenhouse gases in the atmosphere, contributing to a warming planet. This warming, commonly referred to as global warming, has profound effects on natural ecosystems, human health, and the global economy.

One of the most noticeable consequences of climate change is the increasing frequency and intensity of extreme weather events. Hurricanes, droughts, heatwaves, and floods are becoming more common and more severe, leading to devastating consequences for communities worldwide. For instance, recent years have seen unprecedented wildfires in Australia, the United States, and parts of Europe, which have not only destroyed vast areas of forest but also led to significant loss of life and property.

Rising sea levels pose another major threat, particularly for low-lying coastal areas. As polar ice caps and glaciers continue to melt due to rising global temperatures, the sea level rises. Coastal cities, including major metropolitan areas like Miami, New York, and Jakarta, are at risk of becoming uninhabitable in the coming decades if current trends continue. The flooding of coastal areas would lead to the displacement of millions of people, resulting in a global refugee crisis of unprecedented scale.

Beyond environmental impacts, climate change is also affecting global food security. Changes in temperature and precipitation patterns are disrupting traditional agricultural practices, reducing crop yields in some areas and leading to food shortages. For example, many parts of Africa, which are already vulnerable due to poverty and political instability, are experiencing prolonged droughts that are making farming nearly impossible. This exacerbates hunger and malnutrition, particularly among children, and heightens the risk of conflict over scarce resources.

The economic consequences of climate change are also staggering. The damage from extreme weather events, the loss of agricultural productivity, and the cost of adapting infrastructure to cope with rising sea levels and other impacts are projected to cost trillions of dollars over the coming decades. Many developing countries, which contribute the least to greenhouse gas emissions, are expected to suffer the most from these economic impacts, further widening the gap between rich and poor nations.

International efforts to combat climate change have been ongoing for decades, with initiatives like the Paris Agreement aiming to limit global temperature rise to below 2 degrees Celsius above pre-industrial levels. However, progress has been slow, and many experts warn that the window of opportunity to avoid the most catastrophic effects of climate change is rapidly closing. To meet this challenge, countries must drastically reduce their greenhouse gas emissions, invest in renewable energy sources like solar and wind, and implement policies to promote sustainability and resilience.

In conclusion, climate change is one of the most pressing global challenges of our time. Its impacts are far-reaching and multifaceted, affecting not only the environment but also human health, food security, and economic stability. Addressing this challenge requires urgent and coordinated action from governments, businesses, and individuals around the world. Failure to act now could result in irreversible damage to the planet and the quality of life for future generations.

"""
summary = summarize_text(long_text, model, tokenizer)
print("Summary:", summary)


In [ ]:
import evaluate

# Load the ROUGE metric
rouge = evaluate.load("rouge")


def compute_rouge(predictions, references):
    return rouge.compute(predictions=predictions, references=references)

# Example of evaluating a single summary
generated_summary = summarize_text(long_text, model, tokenizer)
reference_summary = "Expected reference summary here."

rouge_scores = compute_rouge([generated_summary], [reference_summary])
print("ROUGE Scores:", rouge_scores)


In [ ]:
!pip install evaluate
